### 📦 Hücre 1 – Ortam Kurulumu ve Sürüm Kontrolü

Bu adımda proje boyunca kullanacağımız temel kütüphaneler yüklenir ve ortamın doğru şekilde hazırlandığından emin olunur.  
Kullanılan kütüphaneler:
- **kaggle**: Veri setini Kaggle API üzerinden indirebilmek için  
- **sentence-transformers**: Metin embedding modellerini yüklemek için  
- **faiss-cpu**: Yüksek hızlı vektör arama ve benzerlik sorgusu için  
- **langchain**: Gerekirse RAG pipeline’ını genişletmek için  
- **gradio**: Web arayüzü oluşturmak için  

Kurulum sonrası, NumPy, Python ve FAISS sürümleri kontrol edilerek ortamın uyumlu olduğu doğrulanır.


In [ ]:
%pip -q install kaggle==1.6.14 sentence-transformers==3.0.1 faiss-cpu==1.12.0 \
                langchain==0.2.14 langchain-community==0.2.12 gradio

import numpy as np, pandas as pd, faiss, sys
print("✅ NumPy:", np.__version__)
print("✅ Python:", sys.version.split()[0])
print("✅ FAISS:", faiss.__version__)


✅ NumPy: 1.26.4
✅ Python: 3.12.12
✅ FAISS: 1.12.0


### 📂 Hücre 2 – TMDB Film Verisinin İndirilmesi

Bu hücrede proje için gerekli olan film verisi **Kaggle** üzerinden çekilir.  
İşlemler adım adım şu şekilde yapılır:

1. Kullanıcının `kaggle.json` API anahtarını yüklemesi istenir.  
2. Anahtar dosyası Colab ortamına yerleştirilip erişim izinleri ayarlanır.  
3. TMDB film verileri (`tmdb_5000_movies.csv` ve `tmdb_5000_credits.csv`) Kaggle’dan indirilir ve `/content/` dizinine çıkarılır.  

Bu veri seti; film başlığı, özet, tür, oyuncular, yönetmen, yazarlar gibi bilgileri içermekte ve öneri motorunun temel verisini oluşturacaktır.


In [ ]:
from google.colab import files
import os, stat

if not os.path.exists('/root/.kaggle/kaggle.json'):
    print("👉 'kaggle.json' yükleyin (Kaggle > Account > Create New Token).")
    up = files.upload()
    assert 'kaggle.json' in up, "kaggle.json seçilmedi."
    os.makedirs('/root/.kaggle', exist_ok=True)
    with open('/root/.kaggle/kaggle.json','wb') as f: f.write(up['kaggle.json'])
    os.chmod('/root/.kaggle/kaggle.json', stat.S_IRUSR | stat.S_IWUSR)

!kaggle datasets download -d tmdb/tmdb-movie-metadata -p /content -q
!unzip -o /content/tmdb-movie-metadata.zip -d /content/ > /dev/null
print("✅ Veri indirildi:", os.path.exists("/content/tmdb_5000_movies.csv"),
      os.path.exists("/content/tmdb_5000_credits.csv"))


👉 'kaggle.json' yükleyin (Kaggle > Account > Create New Token).


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
License(s): other
✅ Veri indirildi: True True


### 🧹 Hücre 3 – Veri Setinin Hazırlanması ve Bilgi Zenginleştirme

Bu aşamada ham veriler okunur, temizlenir ve modelin anlayabileceği bir yapıya dönüştürülür:

- **JSON benzeri sütunlar parse edilir**: türler, anahtar kelimeler, oyuncu kadrosu ve ekip bilgileri Python listelerine çevrilir.
- **Temel bilgiler çıkarılır**: yönetmen, yazarlar ve en popüler oyuncular ayrı sütunlara alınır.
- **Metin temsili oluşturulur (`doc`)**: her film için özet, tür, anahtar kelime, yönetmen ve oyuncuların birleşiminden oluşan tek bir belge metni oluşturulur. Bu belge embedding için kullanılacaktır.
- **Gereksiz sütunlar atılır ve veri temizlenir.**

Bu adımın sonunda her film için anlamlı, semantik olarak zengin bir temsil elde edilir ve bu temsil sonraki embedding aşamasında kullanılacaktır.


In [ ]:
import pandas as pd, ast

movies  = pd.read_csv("/content/tmdb_5000_movies.csv")
credits = pd.read_csv("/content/tmdb_5000_credits.csv")

def parse_json_like(s):
    try: return ast.literal_eval(s) if isinstance(s,str) else []
    except: return []

movies['genres']   = movies['genres'].apply(parse_json_like)
movies['keywords'] = movies['keywords'].apply(parse_json_like)
credits['cast']    = credits['cast'].apply(parse_json_like)
credits['crew']    = credits['crew'].apply(parse_json_like)

df = movies.merge(credits, left_on='id', right_on='movie_id', how='left')

# title güvenceye al
if 'title' not in df.columns:
    if 'title_x' in df.columns: df['title'] = df['title_x']
    elif 'title_y' in df.columns: df['title'] = df['title_y']

def get_director(crew):
    for c in crew or []:
        if c.get('job') == 'Director': return c.get('name')
    return None

def get_writers(crew):
    return [c.get('name') for c in (crew or []) if c.get('department')=='Writing'][:3]

def top_cast(cast, k=5):
    return [c.get('name') for c in (cast or [])][:k]

df['director']      = df['crew'].apply(get_director)
df['writers']       = df['crew'].apply(get_writers)
df['top_cast']      = df['cast'].apply(top_cast)
df['genre_names']   = df['genres'].apply(lambda xs: [x.get('name') for x in xs] if isinstance(xs,list) else [])
df['keyword_names'] = df['keywords'].apply(lambda xs: [x.get('name') for x in xs] if isinstance(xs,list) else [])

def build_doc(row):
    parts=[]
    if isinstance(row.get('overview'), str) and row['overview'].strip(): parts.append(f"🎬 Özet: {row['overview']}")
    if row.get('genre_names'): parts.append(f"📁 Türler: {', '.join(row['genre_names'])}")
    if row.get('keyword_names'): parts.append(f"✨ Anahtar Kelimeler: {', '.join(row['keyword_names'])}")
    if row.get('director'):    parts.append(f"🎥 Yönetmen: {row['director']}")
    if row.get('writers'):     parts.append(f"✍️ Yazarlar: {', '.join(row['writers'])}")
    if row.get('top_cast'):    parts.append(f"⭐ Oyuncular: {', '.join(row['top_cast'])}")
    return "\n".join(parts)

df['doc']  = df.apply(build_doc, axis=1)
df['year'] = df['release_date'].fillna('').str[:4]

keep = ['id','title','year','vote_average','vote_count','popularity','runtime',
        'genre_names','top_cast','director','doc']
df_clean = df[keep].copy()
df_clean = df_clean[df_clean['doc'].str.len()>0].reset_index(drop=True)

print("✅ Kayıt:", len(df_clean))
df_clean.head(2)


✅ Kayıt: 4803


,id,title,year,vote_average,vote_count,popularity,runtime,genre_names,top_cast,director,doc
0,19995,Avatar,2009,7.2,11800,150.437577,162.0,"[Action, Adventure, Fantasy, Science Fiction]","[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron,"🎬 Özet: In the 22nd century, a paraplegic Mari..."
1,285,Pirates of the Caribbean: At World's End,2007,6.9,4500,139.082615,169.0,"[Adventure, Fantasy, Action]","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski,"🎬 Özet: Captain Barbossa, long believed to be ..."


### 🧠 Hücre 4 – Metin Embedding ve FAISS İndeksi Oluşturma

Bu hücrede her film belgesi için çok dilli bir embedding oluşturulur ve bu embedding'ler FAISS vektör arama motoruna eklenir.

- **SentenceTransformer**: `intfloat/multilingual-e5-small` modeli kullanılarak semantik vektörler oluşturulur.
- **Passage formatı**: Belgeler `"passage: ..."` formatına dönüştürülerek modele verilir.
- **FAISS Index**: Kozinüs benzerliği ile çalışan `IndexFlatIP` yapısı oluşturulur ve embedding'ler burada depolanır.
- **Çıktılar kaydedilir**: Hem FAISS indeksi (`movies.index`) hem de meta veriler (`movies.parquet`) diske yazılır.

Bu aşama sayesinde sistem, semantik olarak benzer filmleri yüksek hızla bulabilir hale gelir.


In [ ]:
import numpy as np, faiss
from sentence_transformers import SentenceTransformer

MODEL = "intfloat/multilingual-e5-small"
st = SentenceTransformer(MODEL)

def to_passage(x): return "passage: " + x

texts = df_clean['doc'].tolist()
BATCH=128; embs=[]
for i in range(0,len(texts),BATCH):
    batch = [to_passage(t) for t in texts[i:i+BATCH]]
    vecs = st.encode(batch, normalize_embeddings=True, convert_to_numpy=True)
    embs.append(vecs)
embs = np.vstack(embs).astype('float32')

index = faiss.IndexFlatIP(embs.shape[1])
index.add(embs)

faiss.write_index(index, "/content/movies.index")
df_clean.to_parquet("/content/movies.parquet", index=False)
print("✅ Index:", index.ntotal)


/usr/local/lib/python3.12/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

✅ Index: 4803


### 🔎 Hücre 5 – Sorgu Analizi ve Film Öneri Fonksiyonu

Bu bölümde kullanıcıdan gelen doğal dil sorgular işlenir ve en uygun filmler önerilir. Süreç şu adımlarla gerçekleşir:

1. **Sorgu türü tespiti**: Kullanıcı benzer film mi yoksa tür/tema bazlı arama mı yapıyor, analiz edilir.
2. **Sorgu genişletme (Query Expansion)**:  
   - “Movies like …” gibi sorgular için film bilgileri sorguya eklenir.  
   - Tür bazlı sorgular için ilgili tematik anahtar kelimeler eklenir.
3. **Embedding arama**: Sorgu embedding’i oluşturularak FAISS üzerinde benzer filmler aranır.
4. **Filtreleme**:  
   - IMDb puanı filtresi  
   - Tür filtresi  
   - Tematik kelime filtresi
5. **Hibrit skor hesaplama**:  
   - 0.6: semantik benzerlik  
   - 0.3: IMDb puanı  
   - 0.1: popülerlik  
6. **Sonuçların formatlanması**: En alakalı filmler başlık, yıl, puan, tür ve kısa özetle birlikte döndürülür.

Bu fonksiyon, hem semantik anlamı hem de kaliteyi gözeterek kullanıcıya en uygun film önerilerini sunar.


In [ ]:
import pandas as pd, numpy as np, faiss
from sentence_transformers import SentenceTransformer

st = SentenceTransformer("intfloat/multilingual-e5-small")
index = faiss.read_index("/content/movies.index")
meta  = pd.read_parquet("/content/movies.parquet")

def semantic_search(query, k=50):
    q = st.encode(["query: "+query], normalize_embeddings=True, convert_to_numpy=True).astype('float32')
    D, I = index.search(q, k)
    return I[0], D[0]

def suggest_movies(query, topn=5, min_rating=6.5, genre=None):
    q_lower = query.lower().strip()
    expanded_query = query

    # 🧠 1️⃣ Sorgu tipi tespiti (benzer film mi yoksa tür temelli mi?)
    is_similarity_query = any(phrase in q_lower for phrase in ["movies like", "similar", "like ", "benzeri", "gibi"])

    # 🧠 2️⃣ Query expansion (sadece "movies like" türü sorgular için)
    if is_similarity_query:
        matches = meta[meta['title'].str.lower().apply(lambda x: x in q_lower or x in q_lower.replace("movies like", ""))]
        if len(matches) == 0:
            matches = meta[meta['title'].str.lower().apply(lambda x: any(word in q_lower for word in x.split()))]
        if len(matches) > 0:
            doc_text = matches.iloc[0]['doc']
            expanded_query += " " + doc_text
        expanded_query += " similar style, tone, pacing, themes, and narrative elements"

    # 🧠 3️⃣ Tür / tema temelli sorgular için hafif zenginleştirme
    else:
        if "romantic" in q_lower or "romance" in q_lower:
            expanded_query += " movies about love, dating, relationships, couples, affection"
        if "comedy" in q_lower:
            expanded_query += " funny, humorous, feel-good, light-hearted"
        if "mystery" in q_lower:
            expanded_query += " mystery, investigation, secrets, disappearance, plot twist"
        if "sci-fi" in q_lower or "science fiction" in q_lower:
            expanded_query += " futuristic, space, technology, exploration, dystopia"
        if "slasher" in q_lower:
            expanded_query += " killer, gore, violent, serial killer, horror"

    # 🧠 4️⃣ Embedding araması
    q = st.encode(["query: " + expanded_query], normalize_embeddings=True, convert_to_numpy=True).astype('float32')
    D, I = index.search(q, 100)  # daha fazla aday çek
    df = meta.iloc[I[0]].copy()
    df['similarity'] = D[0]

    # 🧠 5️⃣ Temel filtreler
    if min_rating:
        df = df[df['vote_average'] >= float(min_rating)]

    if genre:
        df = df[df['genre_names'].apply(lambda gs: genre.lower() in [g.lower() for g in gs])]

    # 🧠 6️⃣ Zorunlu tür filtreleme (romantic, comedy vb. varsa)
    if "romantic" in q_lower or "romance" in q_lower:
        df = df[df['genre_names'].apply(lambda gs: any(g.lower() in ["romance", "romantic comedy"] for g in gs))]
    if "comedy" in q_lower:
        df = df[df['genre_names'].apply(lambda gs: "comedy" in [g.lower() for g in gs])]
    if "horror" in q_lower or "slasher" in q_lower:
        df = df[df['genre_names'].apply(lambda gs: "horror" in [g.lower() for g in gs])]
    if "mystery" in q_lower:
        df = df[df['genre_names'].apply(lambda gs: "mystery" in [g.lower() for g in gs] or "thriller" in [g.lower() for g in gs])]
    if "sci-fi" in q_lower or "science fiction" in q_lower:
        df = df[df['genre_names'].apply(lambda gs: "science fiction" in [g.lower() for g in gs])]

    # 🧠 7️⃣ Tematik anahtar kelime filtresi (buraya ekledik ✅)
    thematic_keywords = {
        "romantic": ["love", "relationship", "couple", "dating", "romance", "wedding"],
        "mystery": ["mystery", "detective", "investigation", "murder", "unsolved", "crime", "secrets", "missing", "disappearance"],
        "slasher": ["slasher", "killer", "serial killer", "stab", "blood", "chainsaw"],
        "sci-fi": ["space", "future", "ai", "robot", "time travel", "planet", "exploration", "galaxy"],
        "thriller": ["thriller", "suspense", "cat and mouse", "chase", "tension"],
        "coming-of-age": ["teen", "high school", "youth", "growing up", "adolescence"],
        "post-apocalyptic": ["apocalypse", "end of world", "dystopia", "survivors", "wasteland"],
        "heist": ["robbery", "bank", "crime crew", "plan", "steal"],
        "emotional": ["cry", "tearjerker", "tragic", "sad", "grief", "loss", "death", "moving", "heartbreaking", "weep", "touching", "sentimental"],

    }

    for theme, kws in thematic_keywords.items():
        if theme in q_lower:
            df = df[df['doc'].str.lower().apply(lambda x: any(kw in x for kw in kws))]

    # 🧠 8️⃣ Hibrit skor hesaplama
    df['score'] = (
        0.6 * df['similarity'] +
        0.3 * (df['vote_average'] / 10.0) +
        0.1 * (df['popularity'] / df['popularity'].max())
    )

    df = df.sort_values('score', ascending=False).head(topn)

    # 🧠 9️⃣ Sonuçları formatla
    out = [f"🎯 Sorgu: {query}\n"]
    for i, row in df.iterrows():
        genres = ", ".join(row['genre_names'])
        desc = row['doc'].split("\n")[0].replace("🎬 Özet: ", "")[:220] + "..."
        out.append(f"{len(out)}. {row['title']} ({row['year']}) — ⭐ {row['vote_average']:.1f}\n   🎭 {genres}\n   📖 {desc}\n")

    return "\n".join(out) if len(out) > 1 else "Sonuç bulunamadı."








### 💻 Hücre 6 – Web Arayüzü ile Kullanıcı Etkileşimi

Son aşamada, film öneri motoru bir web arayüzü üzerinden erişilebilir hale getirilir. Bunun için **Gradio** kullanılır.

- Kullanıcıdan alınan parametreler:  
  - 🎬 Sorgu (tema veya benzer film araması)  
  - 🎭 Tür (opsiyonel)  
  - ⭐ Minimum IMDb puanı  
  - 📊 Sonuç sayısı  
- `suggest_movies` fonksiyonu ile öneriler oluşturulur.  
- Sonuçlar, markdown formatında anlaşılır bir şekilde sunulur.

`demo.launch(share=True)` ile uygulama paylaşılabilir hale gelir ve web üzerinden test edilebilir.

Bu arayüz, projenin ürünleşmiş halini temsil eder ve son kullanıcının kolayca öneriler almasını sağlar.


In [1]:
import gradio as gr

def recommend_interface(query, genre, min_rating, topn):
    genre = (genre or "").strip() or None
    return suggest_movies(query, topn=int(topn), min_rating=float(min_rating), genre=genre)

demo = gr.Interface(
    fn=recommend_interface,
    inputs=[
        gr.Textbox(label="🎬 Film Araması", placeholder="ör: romantik komedi, slasher, duygusal bilim kurgu"),
        gr.Textbox(label="🎭 Tür (opsiyonel)", placeholder="ör: Animation, Drama, Horror"),
        gr.Slider(0,10, value=6.5, step=0.1, label="⭐ Minimum Puan"),
        gr.Slider(1,15, value=5, step=1, label="📊 Kaç film?")
    ],
    outputs=gr.Markdown(label="🎞️ Önerilen Filmler"),
    title="🎥 RAG Film Öneri",
    description="E5 embedding + FAISS ile semantik arama, basit filtreler ve temiz çıktı."
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7b633f4107171703c2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
